In [17]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [18]:
# X, y = make_classification(n_samples=99999, n_informative= 4,n_classes=3)
X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

y_train = y_train
y_test = y_test

enc = OneHotEncoder()
y_train_1h = enc.fit_transform(y_train.reshape(-1,1))
y_test_1h = enc.transform(y_test.reshape(-1,1))

In [19]:
X_train

array([[5.7, 3. , 4.2, 1.2],
       [6.2, 3.4, 5.4, 2.3],
       [5.1, 3.8, 1.5, 0.3],
       [7.4, 2.8, 6.1, 1.9],
       [4.7, 3.2, 1.6, 0.2],
       [6.2, 2.9, 4.3, 1.3],
       [5.7, 2.5, 5. , 2. ],
       [6.1, 2.8, 4.7, 1.2],
       [5.5, 2.4, 3.8, 1.1],
       [5.1, 3.7, 1.5, 0.4],
       [6.3, 2.5, 5. , 1.9],
       [4.9, 3.1, 1.5, 0.1],
       [4.8, 3.4, 1.9, 0.2],
       [6. , 2.9, 4.5, 1.5],
       [5.6, 2.8, 4.9, 2. ],
       [7.3, 2.9, 6.3, 1.8],
       [5.9, 3. , 4.2, 1.5],
       [5.3, 3.7, 1.5, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [5.5, 2.4, 3.7, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [5.6, 2.9, 3.6, 1.3],
       [5.6, 3. , 4.5, 1.5],
       [4.6, 3.2, 1.4, 0.2],
       [6.5, 3. , 5.8, 2.2],
       [5.7, 2.9, 4.2, 1.3],
       [5.8, 2.6, 4. , 1.2],
       [4.9, 3.6, 1.4, 0.1],
       [5. , 3.6, 1.4, 0.2],
       [6.3, 2.3, 4.4, 1.3],
       [5. , 3.4, 1.6, 0.4],
       [5.5, 4.2, 1.4, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/home/jonas/anaconda3/envs/brbo/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.98

In [21]:
from simple_model import ClassifierModel, TabularDataset
from torch.utils.data import DataLoader

In [26]:
model = ClassifierModel(input_dim=X_train.shape[1], hidden_dim=16, output_dim=3)
train_dataset = TabularDataset(X_train, y_train)
test_dataset = TabularDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for batch in train_loader:
    print(batch)

[tensor([[6.1000, 2.6000, 5.6000, 1.4000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [5.5000, 2.5000, 4.0000, 1.3000],
        [5.7000, 2.5000, 5.0000, 2.0000],
        [6.0000, 3.0000, 4.8000, 1.8000],
        [5.7000, 2.9000, 4.2000, 1.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.0000, 3.0000, 1.6000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.6000, 3.2000, 1.4000, 0.2000],
        [6.8000, 3.2000, 5.9000, 2.3000],
        [7.7000, 3.8000, 6.7000, 2.2000],
        [6.2000, 3.4000, 5.4000, 2.3000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [4.8000, 3.0000, 1.4000, 0.3000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [4.5000, 2.3000, 1.3000, 0.3000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.8000, 2.8000, 4.8000, 1.4000],
        [4.4000, 3.2000, 1.3000, 0.2000],
        [4.8000, 3.4000, 1.9000, 

In [23]:
print(model)

ClassifierModel(
  (input): Linear(in_features=4, out_features=16, bias=True)
  (hidden): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
)


In [24]:
model.fit(train_dataset)

tensor([1])


TypeError: unsqueeze() missing 1 required positional arguments: "dim"

In [ ]:
import torch
import torch.nn as nn

# Example
batch_size = 4
num_classes = 3

# Random predictions (logits)
logits = torch.randn(batch_size, num_classes)

# Random target labels
labels = torch.tensor([0, 2, 1, 0])

criterion = nn.CrossEntropyLoss()
loss = criterion(logits, labels)
print(loss)
print(labels)
print(logits)


tensor(1.6076)
tensor([0, 2, 1, 0])
tensor([[-1.3542,  0.1611, -0.5087],
        [-0.4759,  1.1457,  1.0074],
        [-1.1535, -1.5654,  0.9661],
        [ 0.0767,  0.1515, -2.0423]])


In [25]:
from sklearn.datasets import load_iris
import torch
from torch.utils.data import Dataset, DataLoader

# Step 1: Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Step 2: Convert the dataset to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Step 3: Create a custom Dataset class
class IrisDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        sample = {
            'feature': self.features[idx],
            'label': self.labels[idx]
        }
        return sample

# Step 4: Use the DataLoader
iris_dataset = IrisDataset(X_tensor, y_tensor)
batch_size = 16
data_loader = DataLoader(dataset=iris_dataset, batch_size=batch_size, shuffle=True)

# Example of iterating through the DataLoader
for batch in data_loader:
    print(batch)


{'feature': tensor([[5.8000, 2.7000, 5.1000, 1.9000],
        [5.4000, 3.4000, 1.7000, 0.2000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.4000, 3.7000, 1.5000, 0.2000],
        [6.0000, 2.9000, 4.5000, 1.5000],
        [5.6000, 3.0000, 4.1000, 1.3000],
        [6.3000, 2.5000, 5.0000, 1.9000],
        [6.4000, 3.1000, 5.5000, 1.8000],
        [5.0000, 3.2000, 1.2000, 0.2000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [5.0000, 3.4000, 1.6000, 0.4000],
        [6.3000, 2.8000, 5.1000, 1.5000],
        [5.9000, 3.0000, 5.1000, 1.8000],
        [5.3000, 3.7000, 1.5000, 0.2000],
        [5.6000, 2.5000, 3.9000, 1.1000],
        [4.5000, 2.3000, 1.3000, 0.3000]]), 'label': tensor([2, 0, 2, 0, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 0])}
{'feature': tensor([[4.9000, 3.6000, 1.4000, 0.1000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [5.2000, 2.7000, 3.9000, 1.4000],
        [6.1000, 2.8000, 4.0000, 1.3000],
        [6.4000, 3.2000, 4.5000, 1.5000],
        [6.3000, 2.3000, 